# LArCV Labeling

The reconstruction chain operates on a specific type of data files: [LArCV](https://github.com/LArbys/LArCV) (Liquid Argon Computer Vision). These files are ROOT files with basic data representations relevant to doing machine learning (ML) in LArTPCs. They contain only the necessary information for ML-based reconstruction algorithms and can be loaded fast, so as to not restrict the execution speed.

These files are produced from either:
- `stage1` files (with SpacePoint data products) files in LArSoft (DUNE-FD and its prototypes)
- `flow` files (with prompt hits data products) files in ndlar-flow (DUNE-ND and its prototypes)

The follow up three tutorials (tomorrow) will show you how to produce these files. This tutorial will focus on the content of these files.

***
***
## I. Browsing a LArCV file using ROOT

The basic container used in this workshop contains the necessary ROOT and LArCV dependencies. If you are running this locally, you will need to build LArCV2 (requires ROOT): [https://github.com/DeepLearnPhysics/larcv2](https://github.com/DeepLearnPhysics/larcv2)

There are multiple ways to look at the content of a LArCV ROOT file:
1. Use TBrowser
```shell
$ rootbrowse file.root
```
2. Load file in a ROOT interactive shell
```shell
$ root -l
root [0] f = TFile("file.root");
root [1] f.ls()
```
3. Load file using PyROOT

4. Load file using [uproot](https://uproot.readthedocs.io/en/latest/basic.html)

Let's take a look at the contents of a `LArCV` simulation file using PyROOT:

In [ ]:
import larcv
import ROOT

fname = '/global/cfs/cdirs/m5252/dune/spine/workshop/larcv/protodune-vd_small.root' # Change this path if you are not on NERSC
#fname = 'LOCAL_PATH'
f = ROOT.TFile(fname)
f.ls()

In [ ]:
import uproot

tree = uproot.open(fname)
tree.keys()

There are several data types, as you can see:
- `sparse3d_*` refers to a [larcv::EventSparseTensor3D](https://github.com/DeepLearnPhysics/larcv2/blob/develop/larcv/core/DataFormat/EventVoxel3D.h#L47), a tensor of voxel coordinates
  - Each `SparseTensor3D` is a list of `(voxel_id, value)` pairs which can map to a list of `(x, y, z, value)`
  - E.g. charge per voxel, semantic label, cluster label, etc.
- `cluster3d_*` refers to a [larcv::EventClusterVoxel3D](https://github.com/DeepLearnPhysics/larcv2/blob/develop/larcv/core/DataFormat/EventVoxel3D.h#L27), a list of tensor coordinates
  - Each `ClusterVoxel3D` object contains a list of SparseTensor3D, typically one per particle
- `particle_*` refers to a [larcv::EventParticle](https://github.com/DeepLearnPhysics/larcv2/blob/develop/larcv/core/DataFormat/EventParticle.h#L26), a list of [larcv::Particle](https://github.com/DeepLearnPhysics/larcv2/blob/develop/larcv/core/DataFormat/Particle.h#L27) objects
  - Each `Particle` object has a bunch of MC truth particle level attributes
  - E.g. start/end points, PDG code, creation process, etc.
- `neutrino_*` refers to a [larcv::EventNeutrino](https://github.com/DeepLearnPhysics/larcv2/blob/develop/larcv/core/DataFormat/EventNeutrino.h#L26), a list of [larcv::Neutrino](https://github.com/DeepLearnPhysics/larcv2/blob/develop/larcv/core/DataFormat/Neutrino.h#L27) objects
  - Each `Neutrino` object has a bunch  of MC truth neutrino level attributes
  - E.g. vertex, interaction type (CC, NC), neutrino energy, etc.
- `opflash_*` refers to a [larcv::Flash](https://github.com/DeepLearnPhysics/larcv2/blob/develop/larcv/core/DataFormat/Flash.h#L25)
  - Dumps out the list of flash and their associated information
  - E.g. PE sum in each PMT, flash time, etc.
- `crthit_*` refers to a [larcv::CRTHit](https://github.com/DeepLearnPhysics/larcv2/blob/develop/larcv/core/DataFormat/CRTHit.h#L28)
  - Dumps out the list of CRT hits and their associated information
  - E.g. PE sum in the hit, hit time, location of the CRT plane, etc.
- `trigger_*` refers to a [larcv::Trigger](https://github.com/DeepLearnPhysics/larcv2/blob/develop/larcv/core/DataFormat/Trigger.h)
  - Contains trigger information (index, time)

***
### Check content

There are ways to check on the content of the LArCV file, for instance one can dump the size of each cluster of points in each event (image), i.e. the size of each particle, following this procedure:

In [ ]:
import numpy as np
tree = f.Get("cluster3d_pcluster_tree")
print('Number of entries', tree.GetEntries())

for entry in range(tree.GetEntries()):
    tree.GetEntry(entry)
    event = tree.cluster3d_pcluster_branch
    clusters = event.as_vector()
    print("Number of clusters = ", len(clusters))
    cluster_sizes = []
    for c in clusters:
        cluster_sizes.append(c.as_vector().size())
    print(np.array(cluster_sizes))

Now close the file:

In [ ]:
f.Close()

***
***
## II. Browsing the LArCV file using `SPINE`

Before starting anything, it is good practice to look at your dataset in an event display. This chapter is strictly about the I/O part of SPINE (independently of everything else, models, etc) and how to use it to visualize your dataset.

Two options to gain access to SPINE (not pre-packaged in the image):
- `pip install`
- Point to a local install at NERSC

In [ ]:
# import sys
# Set software directory (alternative)
# SOFTWARE_DIR = '/global/cfs/cdirs/m5252/software/spine/src/' # Change this path if you are not on NERSC. This can work instead of pip install
# sys.path.insert(0, SOFTWARE_DIR)

%pip install --user --upgrade spine-ml

In [ ]:
import spine

Set up path to the data directory

In [ ]:
DATA_DIR = '/global/cfs/cdirs/m5252/dune/spine/workshop/larcv/' # Change this path if you are not on NERSC

## Configuration
You need to specify a *configuration*, in YAML syntax, which tells `SPINE` how you want to access the data: how many images (`batch_size`), the path to your dataset, which quantities you want to retrieve from the dataset. You can even limit the I/O to a specific list of entry numbers using `entry_list`, or skip a list with `skip_entry_list`

In [ ]:
DETECTOR = 'protodune-vd'
DATA_PATH = DATA_DIR+'/protodune-vd_small.root'

cfg = """
base:
  verbosity: info
geo:
  detector: DETECTOR
io:
  loader:
    batch_size: 4
    shuffle: False
    num_workers: 4
    collate_fn: all
    dataset:
      name: larcv
      file_keys: DATA_PATH
      limit_num_files: 10
      #entry_list: [6436, 562, 3802, 6175, 15256] # can also be specified as a file
      #skip_entry_list: [12, 354] # can also be specified as a file
      schema:
        input_data:
          parser: sparse3d
          sparse_event: sparse3d_pcluster
        seg_label:
          parser: sparse3d
          sparse_event: sparse3d_pcluster_semantics
        clust_label:
          parser: cluster3d
          cluster_event: cluster3d_pcluster
          particle_event: particle_corrected
          sparse_semantics_event: sparse3d_pcluster_semantics
          sparse_value_event: sparse3d_pcluster
          add_particle_info: true
          clean_data: true
        ppn_label:
          parser: particle_points
          particle_event: particle_corrected
          sparse_event: sparse3d_pcluster
        meta:
          parser: meta
          sparse_event: sparse3d_pcluster
        run_info:
          parser: meta
          sparse_event: sparse3d_pcluster
""".replace('DETECTOR', DETECTOR).replace('DATA_PATH', DATA_PATH)

print(cfg)

`SPINE` expects a ROOT file (created with LArCV, a C++ library to process
LArTPC images) as input file. You can explore this file with ROOT alone, if you know
how to use ROOT, but the most intuitive way to visualize it is to use the I/O module
of `SPINE`.

### What are parsers?
Images are stored in ROOT + LArCV format. Parsers are classes in `SPINE`
that read the information stored in the ROOT file, select just what we are interested
in (e.g. particle information ? energy depositions ?) and format it in a friendly way
for our chain.

They only need to know the names of certain quantities stored in the ROOT file (as TTree,
if you know what this is).

### Base configuration - a brief gist

The base configuration block generally instructs the driver as to how to behave in general
(logging, verbosity, etc.). We'll see a lot more of this block later, but for now I will
just highlight the verbosity flag, which allows one to set the amount of information reported:
- `debug`: Show every logging message coming from SPINE
- `info`: Show only basic information messages
- `warning`: Only show warnings
- `error`: Only show error messages
- `fatal`: Only show fatal error messages

### I/O Configuration - a brief gist

`SPINE` uses the YAML format for its configuration. Here is a skeleton config
that shows only the parameters that will matter the most for you:
```
io:
  loader:
    batch_size: 16
    (...)
    sampler: random_sequence
    (...)
    dataset:
      (...)
      file_keys: DATA_PATH
      (...)
      schema:
        input_data:
          parser: sparse3d
          sparse_event: sparse3d_pcluster
        (...)
```

The main things to pay attention to in this data I/O configuration block are:
* `batch_size` : number of images loaded per batch
* `sampler`: randomization (default: none if sampler is commented out, enabled if you include the RandomSequenceSampler, named random_sequence)
* `file_keys`: dataset filename(s). Can be either
  * A single file path (supports wildcards)
  * A list of file paths (supports wildcards)
  * A path to a text file which contains a list of file paths (`\n` separated)
* `schema`: list of data parsers and their individual configurations

The schema has a simple format. It has two main components:
 * `parser`: parser name
 * `(key, value)` pairs which correspond to arguments of the parser `process` function.

```yaml
	my_custom_data_name:
	  parser: whatever # this is the parser name
	  arg_event: sparse3d_pcluster # this string will be the parser's `arg_event` argument
	  arg_1: false # this string will be the parser's `arg_1` argument
	  # etc
```

The parser's arguments that end with `_event` are the names of quantities stored in the input ROOT file.

A real life example would look like this:

```yaml
      input_data:
        parser: sparse3d
        sparse_event: sparse3d_pcluster
```

This tells us that we want a field called `input_data` (our choice) in the input data
dictionary. For the sake of example let's call this input dictionary `data`.
The parser name needs to be the first in the list that follows. Hence,
`data['input_data']` will be the output of the `sparse3d` parser .
That parser will receive as arguments whatever names follow in the list - here, there
is just one, `sparse3d_pcluster`.

You can have as many such items in the schema list - each of them will be available
in the input data dictionary under the name that you specify.

Note: some stages of the chain might expect a specific name in the input dictionary.
Sometimes this is configurable in the network configuration.

These are typical inputs that you would be looking at:
* `sparse3d` gets you the actual 3D image, the voxels and various values for each voxel. For each voxel it will parse as many features as you are providing branch names. Here each voxel in `input_data` will have a single feature coming from `sparse3d_pcluster` (the true energy depositions). Each voxel in `seg_label` will have a single feature coming from `sparse3d_pcluster_semantics` (which holds the true semantic labels).
* `cluster3d` is parsing particle-wise information and aggregating it. For each non-zero voxel it will provide you with energy deposit, cluster id, interaction id, neutrino vs cosmic label, etc.
* `particle_points` retrieves the coordinates and semantic class of points of interest for PPN.

Now that the configuration is defined, you can feed it to `SPINE`:

In [ ]:
import yaml
from spine.driver import Driver

cfg = yaml.safe_load(cfg)

# prepare function configures necessary "handlers"
driver = Driver(cfg)

## Iterate
One way to iterate through the dataset is to call the `process` function of the driver:

In [ ]:
# driver.apply_filter(entry_list=[0, 1, 2, 3])
data = driver.process()

You can see that `data` is a dictionary whose keys match the names specified in the configuration block above:

In [ ]:
data.keys()

As we use a batch loader, each of these objects corresponds to multiple entries in the dataset, e.g.

In [ ]:
data['index']

The tensor objects are stored in `TensorBatch` objects which contain functions to quickly slice the tensor between the different entries that make it up. E.g.:

In [ ]:
data['clust_label']

`clust_label` is made up of 4 entries. Two ways to access it's content:
- `data['clust_label'].tensor` returns the full collated tensor of coordinates/values
- `data['clust_label'][i]` returns the tensor corresponding to the i^th entry in the batch

Let us visualize the distribution of pixel values in the cluster label tensor

In [ ]:
from spine.utils.globals import VALUE_COL
from matplotlib import pyplot as plt

plt.hist(data['clust_label'].tensor[:, VALUE_COL], range=[0,100], bins=50, histtype='step', linewidth=2)
plt.xlabel('Energy [MeV]')
plt.ylabel('Space points')
plt.grid()
plt.show()

Little segway about a nice tool few nice tools:
- `%pdef` prints out a function definition
- `%pdoc` prints out a docstring (equivalent to python's `help` command)
- `%pfile` prints out the entire source code of a specific python module

Let's use the last one to check out our globals

In [ ]:
import spine.utils.globals

%pfile spine.utils.globals

## Taking a look at the images

It is time to run this configuration and see what we get out of it! We use Plotly to visualize the 3D images.

In [ ]:
entry = 0

Let's select the second entry.

In [ ]:
from spine.utils.globals import SHAPE_COL

clust_label = data['clust_label'][entry]
input_data = data['input_data'][entry]
seg_label = data['seg_label'][entry][:, SHAPE_COL]
ppn_label = data['ppn_label'][entry]

### Input data
Let us visualize the `input_data` first:

These are the energy deposits detected by the LArTPC. The energy scale might be true if you are looking at true labels,
or it might be a reconstructed energy, depending on what you are loading into `input_data`.

In [ ]:
from plotly import graph_objs as go
from plotly.offline import iplot

from spine.vis.geo import GeoDrawer
from spine.vis.point import scatter_points
from spine.vis.layout import layout3d

from spine.utils.globals import GHOST_SHP

geo_drawer = GeoDrawer(detector_coords=False)
tpc_traces = geo_drawer.tpc_traces(meta=data['meta'][entry])
opt_traces = geo_drawer.optical_traces(meta=data['meta'][entry])
crt_traces = geo_drawer.crt_traces(meta=data['meta'][entry])

nonghost_mask = seg_label < GHOST_SHP

trace = []

trace += scatter_points(input_data, color=input_data[:, VALUE_COL],
                        markersize=2, cmin=0, cmax=50, colorscale='Inferno', name='Input charge')

trace += scatter_points(input_data[nonghost_mask],
                        color=input_data[nonghost_mask, VALUE_COL],
                        markersize=2, cmin=0, cmax=50, colorscale='Inferno', name='Input charge (true non-ghost)')

trace += tpc_traces
trace += opt_traces
trace += crt_traces

fig = go.Figure(data=trace, layout=layout3d(use_geo=True, detector_coords=False, meta=data['meta'][entry], show_crt=True))

iplot(fig)

What can we do with scatter_points?

In [ ]:
%pdoc scatter_points
# help(scatter_points)

### Semantic labels
Let us look at the semantic labels next: for each voxel, there is a class label which takes integer values in `[0, 4]`.

In [ ]:
from spine.utils.globals import PPN_LTYPE_COL
from spine.vis.layout import PLOTLY_COLORS

trace = []

trace+= scatter_points(input_data, color=seg_label,
                       markersize=2, cmin=0, cmax=5, colorscale=PLOTLY_COLORS[:6],
                       name='Seg. labels (no ghosts)')

trace += tpc_traces

fig = go.Figure(data=trace,layout=layout3d(meta=data['meta'][entry]))

iplot(fig)

### Points of interest
`ppn_label` holds the coordinates of points of interest. They are displayed as big dots in the visualization.
The dots color corresponds to the true semantic class attributed to each point of interest.

In [ ]:
from spine.utils.globals import PPN_LTYPE_COL
from spine.vis.layout import PLOTLY_COLORS

trace = []

trace+= scatter_points(input_data, color=seg_label,
                       markersize=1, cmin=0, cmax=5, colorscale=PLOTLY_COLORS[:6],
                       name='Seg. labels (no ghosts)')

trace += scatter_points(ppn_label, color=ppn_label[:, PPN_LTYPE_COL],
                        markersize=5, cmin=0, cmax=5, colorscale=PLOTLY_COLORS[:6])
trace[-1].name = "True point labels"

trace += tpc_traces

fig = go.Figure(data=trace,layout=layout3d(meta=data['meta'][entry]))

iplot(fig)

### Particle instances
A particle instance is a cluster of voxels that belong to an individual particle. Each color here represents
a different particle instance.

In [ ]:
from spine.utils.globals import GROUP_COL
from spine.vis.layout import HIGH_CONTRAST_COLORS

trace = []

trace+= scatter_points(clust_label, color=clust_label[:, GROUP_COL],
                       markersize=2, cmin=0, cmax=50, colorscale=HIGH_CONTRAST_COLORS,
                       name='True group labels')

trace += tpc_traces

fig = go.Figure(data=trace,layout=layout3d(meta=data['meta'][entry]))

iplot(fig)

### Interaction groups
Particle instances can then be grouped into *interactions*. Each color is a different
interaction in this visualization.

In [ ]:
from spine.utils.globals import INTER_COL

trace = []

trace+= scatter_points(clust_label,color=clust_label[:, INTER_COL],
                        markersize=2, cmin=0, cmax=50, colorscale=HIGH_CONTRAST_COLORS)
trace[-1].name = 'True interaction labels'

trace += tpc_traces

fig = go.Figure(data=trace,layout=layout3d(meta=data['meta'][entry]))

iplot(fig)

### Neutrino vs cosmics
There are two types of interactions: the ones due to a neutrino traversing the
detector volume (i.e. our signal!), and the ones due to cosmic rays (background).

In [ ]:
from spine.utils.globals import NU_COL

trace = []

trace += scatter_points(clust_label, color=clust_label[:, NU_COL],
                        markersize=2, cmin=-1, cmax=0, colorscale='Portland',
                        name='Nu / cosmic labels')

trace += tpc_traces

fig = go.Figure(data=trace,layout=layout3d(meta=data['meta'][entry]))

iplot(fig)

***
***
## III. Exercises

If you wanna experiment a bit, here's a few suggestions of what to look at:
- Using PyROOT, check the number of voxels in a sparse tensor object
- Using PyROOT, select the first Particle object in the list, use python `help` to understand its content
  - What is the first particle's type? Initial energy? Deposited energy? Does it have children?
- Using PyROOT, select the first Neutrino object in the list, use python `help` to understand its content
  - What type of neutrino interacted? What was its energy? What was the interaction process?
- Check out the remaining available label columns in the `cluster_label` tensor (PID_COL, PGRP_COL, etc.). Do you understand what they represent and do you think they correspond to your expectations?
- The parser definitions live under `spine.io.parse.[sparse, cluster, particle, misc]`. Use %pdoc or help to find out what this parser does and the argument it needs, then try to add parsers to the inline configuration and visualize the output, e.g.
  - `sparse3d_ghost`
  - `particles`
  - `meta`

In [ ]:
from spine.io.torch.dataset import PARSER_DICT # Need to expose this more easily

print(PARSER_DICT.keys())
help(PARSER_DICT['sparse3d_ghost'])